# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 31 2022 10:30AM,245419,15,30010324,"Alliance Pharma, Inc.",Released
1,Aug 31 2022 10:30AM,245419,15,30010325,"Alliance Pharma, Inc.",Released
2,Aug 31 2022 10:30AM,245419,15,30010326,"Alliance Pharma, Inc.",Released
3,Aug 31 2022 10:30AM,245419,15,30010328,"Alliance Pharma, Inc.",Released
4,Aug 31 2022 10:30AM,245419,15,30010327,"Alliance Pharma, Inc.",Released
5,Aug 31 2022 10:30AM,245419,15,30010329,"Alliance Pharma, Inc.",Released
6,Aug 31 2022 10:30AM,245419,15,30010330,"Alliance Pharma, Inc.",Released
7,Aug 31 2022 10:30AM,245419,15,30010332,"Alliance Pharma, Inc.",Released
8,Aug 31 2022 10:30AM,245419,15,30010334,"Alliance Pharma, Inc.",Released
9,Aug 31 2022 10:30AM,245419,15,30010333,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,245413,Released,1
26,245416,Executing,1
27,245417,Released,2
28,245418,Executing,1
29,245419,Released,30


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
245413,NaN,NaN,1.0
245416,NaN,1.0,NaN
245417,NaN,NaN,2.0
245418,NaN,1.0,NaN
245419,NaN,NaN,30.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244946,0.0,0.0,66.0
245276,0.0,0.0,1.0
245277,0.0,0.0,1.0
245280,0.0,1.0,0.0
245281,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244946,0,0,66
245276,0,0,1
245277,0,0,1
245280,0,1,0
245281,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244946,0,0,66
1,245276,0,0,1
2,245277,0,0,1
3,245280,0,1,0
4,245281,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244946,,,66
1,245276,,,1
2,245277,,,1
3,245280,,1,
4,245281,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 31 2022 10:30AM,245419,15,"Alliance Pharma, Inc."
30,Aug 31 2022 10:12AM,245418,19,"AdvaGen Pharma, Ltd"
31,Aug 31 2022 10:10AM,245417,10,"Methapharm, Inc."
33,Aug 31 2022 10:08AM,245416,21,"NBTY Global, Inc."
34,Aug 31 2022 9:59AM,245403,20,Alumier Labs Inc.
56,Aug 31 2022 9:56AM,245413,10,"Methapharm, Inc."
57,Aug 31 2022 9:33AM,245412,12,Hush Hush
67,Aug 31 2022 9:11AM,245411,10,"Senseonics, Incorporated"
68,Aug 31 2022 9:09AM,245410,16,Sartorius Bioprocess Solutions
70,Aug 31 2022 9:08AM,245409,16,Sartorius Lab Products and Service


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 31 2022 10:30AM,245419,15,"Alliance Pharma, Inc.",,,30
1,Aug 31 2022 10:12AM,245418,19,"AdvaGen Pharma, Ltd",,1,
2,Aug 31 2022 10:10AM,245417,10,"Methapharm, Inc.",,,2
3,Aug 31 2022 10:08AM,245416,21,"NBTY Global, Inc.",,1,
4,Aug 31 2022 9:59AM,245403,20,Alumier Labs Inc.,,,22
5,Aug 31 2022 9:56AM,245413,10,"Methapharm, Inc.",,,1
6,Aug 31 2022 9:33AM,245412,12,Hush Hush,,10,
7,Aug 31 2022 9:11AM,245411,10,"Senseonics, Incorporated",,,1
8,Aug 31 2022 9:09AM,245410,16,Sartorius Bioprocess Solutions,,,2
9,Aug 31 2022 9:08AM,245409,16,Sartorius Lab Products and Service,,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 31 2022 10:30AM,245419,15,"Alliance Pharma, Inc.",30,,
1,Aug 31 2022 10:12AM,245418,19,"AdvaGen Pharma, Ltd",,1,
2,Aug 31 2022 10:10AM,245417,10,"Methapharm, Inc.",2,,
3,Aug 31 2022 10:08AM,245416,21,"NBTY Global, Inc.",,1,
4,Aug 31 2022 9:59AM,245403,20,Alumier Labs Inc.,22,,
5,Aug 31 2022 9:56AM,245413,10,"Methapharm, Inc.",1,,
6,Aug 31 2022 9:33AM,245412,12,Hush Hush,,10,
7,Aug 31 2022 9:11AM,245411,10,"Senseonics, Incorporated",1,,
8,Aug 31 2022 9:09AM,245410,16,Sartorius Bioprocess Solutions,2,,
9,Aug 31 2022 9:08AM,245409,16,Sartorius Lab Products and Service,2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 31 2022 10:30AM,245419,15,"Alliance Pharma, Inc.",30,,
1,Aug 31 2022 10:12AM,245418,19,"AdvaGen Pharma, Ltd",,1,
2,Aug 31 2022 10:10AM,245417,10,"Methapharm, Inc.",2,,
3,Aug 31 2022 10:08AM,245416,21,"NBTY Global, Inc.",,1,
4,Aug 31 2022 9:59AM,245403,20,Alumier Labs Inc.,22,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 31 2022 10:30AM,245419,15,"Alliance Pharma, Inc.",30.0,NaN,NaN
1,Aug 31 2022 10:12AM,245418,19,"AdvaGen Pharma, Ltd",NaN,1.0,NaN
2,Aug 31 2022 10:10AM,245417,10,"Methapharm, Inc.",2.0,NaN,NaN
3,Aug 31 2022 10:08AM,245416,21,"NBTY Global, Inc.",NaN,1.0,NaN
4,Aug 31 2022 9:59AM,245403,20,Alumier Labs Inc.,22.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 31 2022 10:30AM,245419,15,"Alliance Pharma, Inc.",30.0,0.0,0.0
1,Aug 31 2022 10:12AM,245418,19,"AdvaGen Pharma, Ltd",0.0,1.0,0.0
2,Aug 31 2022 10:10AM,245417,10,"Methapharm, Inc.",2.0,0.0,0.0
3,Aug 31 2022 10:08AM,245416,21,"NBTY Global, Inc.",0.0,1.0,0.0
4,Aug 31 2022 9:59AM,245403,20,Alumier Labs Inc.,22.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1717796,32.0,13.0,0.0
12,245412,0.0,10.0,0.0
15,735701,108.0,0.0,34.0
16,736227,4.0,1.0,0.0
19,245418,0.0,1.0,0.0
20,490742,51.0,0.0,0.0
21,2207755,6.0,3.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1717796,32.0,13.0,0.0
1,12,245412,0.0,10.0,0.0
2,15,735701,108.0,0.0,34.0
3,16,736227,4.0,1.0,0.0
4,19,245418,0.0,1.0,0.0
5,20,490742,51.0,0.0,0.0
6,21,2207755,6.0,3.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,32.0,13.0,0.0
1,12,0.0,10.0,0.0
2,15,108.0,0.0,34.0
3,16,4.0,1.0,0.0
4,19,0.0,1.0,0.0
5,20,51.0,0.0,0.0
6,21,6.0,3.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,32.0
1,12,Released,0.0
2,15,Released,108.0
3,16,Released,4.0
4,19,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,0.0,0.0,34.0,0.0,0.0,0.0,0.0
Executing,13.0,10.0,0.0,1.0,1.0,0.0,3.0
Released,32.0,0.0,108.0,4.0,0.0,51.0,6.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,34.0,0.0,0.0,0.0,0.0
1,Executing,13.0,10.0,0.0,1.0,1.0,0.0,3.0
2,Released,32.0,0.0,108.0,4.0,0.0,51.0,6.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,34.0,0.0,0.0,0.0,0.0
1,Executing,13.0,10.0,0.0,1.0,1.0,0.0,3.0
2,Released,32.0,0.0,108.0,4.0,0.0,51.0,6.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()